To run this program you have two options: upload a single czi image (preferred), or upload a fixed size square MT-Channel png/jpg (eg. 120x120 image). The only time when you should upload a single czi file is if the image is already zoomed into the area that you want to measure with this program with the nuclei of interest in the center. Otherwise, passing in an entire muscle image will cause issues with miscellaneous nuclei above and below the muscle that will be detected. To upload the image(s), simply run the corresponding loading code chunk below based on the image type you are choosing.

In [ ]:
!pip install aicsimageio
!pip install aicspylibczi
import numpy as np
from keras.preprocessing import image
import keras.utils as ut
import cv2
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from PIL import Image, ImageDraw
from skimage.measure import label
from aicsimageio import AICSImage
from aicsimageio.readers import CziReader
import os
import pandas as pd
from scipy.ndimage import binary_dilation, binary_fill_holes
from scipy import ndimage as ndi
from skimage.feature import peak_local_max


In [ ]:
dfo = '/content/Concentric Circle Data.xlsx'

## Helper Functions

In [ ]:
def crop_black_border(image):
  # remove the black border
  row_max = image.max(axis = 1)
  row_ind = np.where(row_max>0)[0]
  col_max = image.max(axis = 0)
  col_ind = np.where(col_max>0)[0]
  return image[row_ind[0] : row_ind[-1]+1, col_ind[0] : col_ind[-1]+1]

def OtsuTwoLevelThreshold(th_high, th_low):
  plt.figure(figsize = (10,10))
  im_o = micro_im.copy()
  im_o[im_o > th_high] = 0
  im_o[im_o < th_low] = 0

  plt.subplot(121)
  plt.title('Input')
  plt.imshow(micro_im, cmap = 'gray')

  plt.subplot(122)
  plt.title('Output')
  plt.imshow(im_o, cmap = 'gray')
  print('Selected Param: th = (%d, %d)' % (th_high, th_low))
  return

def imAdjust(I, thres=[1,99]):
    # compute percentile: remove too big or too small values
    I_low, I_high = np.percentile(I.reshape(-1), thres)
    # thresholding
    I[I > I_high] = I_high
    I[I < I_low] = I_low
    # scale to 0-1
    I = (I.astype(float)- I_low)/ (I_high-I_low)
    # convert it to uint8
    I = (I * 255).astype(np.uint8)
    return I

## Loading czi images for nuclei detection (Preferred)

To load czi images into this code, in your "MyDrive" of your google drive, create a folder called "Concentric_Data" (if you haven't done so already). Then for each experiment/folder of images you want to analyze, upload the folder containing those czi images to "Concentric_Data". Then in the code below, change the genotype_name variable to the name of the folder you uploaded to "Concentric_Data". Spelling and capitalization matter.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base_directory = '/content/drive/MyDrive/Concentric_Data'

genotype_name = 'Concentric circle'
def read_and_store_images(sub_folder):
    folder_path = os.path.join(base_directory, sub_folder)
    micro = []
    nuclei = []
    name = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.czi'):
            file_path = os.path.join(folder_path, filename)
            img = AICSImage(file_path)
            nuclei_im = img.get_image_data("ZYX", C=1)
            micro_im = img.get_image_data("ZYX", C=0)

            nuclei_im_max = np.max(nuclei_im, axis=0)
            micro_im_max = np.max(micro_im, axis=0)

            shape = nuclei_im_max.shape
            nuclei_im_max = cv2.rotate(nuclei_im_max, cv2.ROTATE_90_COUNTERCLOCKWISE)
            micro_im_max = cv2.rotate(micro_im_max, cv2.ROTATE_90_COUNTERCLOCKWISE)
            name.append(filename.rstrip('.czi'))
            micro.append(micro_im_max)
            nuclei.append(nuclei_im_max)
    return micro, nuclei, name

micro_im, nuclei_im, file_name = read_and_store_images(genotype_name)

## Loading single MT channel image jpg/png

Only run this code if you want to run concentric analysis with jpg or png images of zoomed in microtubule images.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base_directory = '/content/drive/MyDrive'

def load_im(sub_folder):
  folder_path = os.path.join(base_directory, sub_folder)
  micro = []
  for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    nuclei_im = ut.load_img('/content/C3-MAX_VL3 4.3-1.jpg', grayscale=True)
    micro_im = ut.load_img(file_path, grayscale=True)

    nuclei_im = ut.img_to_array(nuclei_im, dtype='uint8')
    micro_im = ut.img_to_array(micro_im, dtype='uint8')

    nuclei_im = nuclei_im[:,:,0]
    micro_im = micro_im[:,:,0]

    nuclei_im = crop_black_border(nuclei_im)
    micro_im = crop_black_border(micro_im)
    if micro_im.shape[0] != 80 or micro_im.shape[1] != 80:
      print(file_path)
    micro.append(micro_im)
  return micro

#Change name in parenthesis to the name of folder on google drive
micro_im = load_im('folder name')

## Visualizing each loaded MT-channel image

In [ ]:
for im in micro_im:
  im_adjust = imAdjust(im)
  plt.imshow(im_adjust, cmap = 'gray')
  print(im_adjust.dtype)
  plt.show()

## Automatically Calculating Nuclei Center for Airyscan only (CZI)

The following code will loop through all the input images and(hopefully) automatically calculate the center of the nuclei. The calculated centers will be displayed for human check. Of course, there is no algorithm that works 100%, so if it makes a mistake, you can always manually type in the x,y coordinates of the nuclei centers by uncommenting the last two lines of code below (simply delete the # sign for uncommenting a line of code)

In [ ]:
y_centers = []
x_centers = []
for image in nuclei_im:
  im = imAdjust(image)

  n_image_mask = cv2.GaussianBlur(im,(85,85),0)
  plt.imshow(n_image_mask, cmap = 'gray')
  plt.title('Blurred nuclei image')
  plt.show()
  threshold = 50
  bin_n_image = n_image_mask > threshold

  dilated_image = binary_dilation(bin_n_image, structure=np.ones((35, 35)))
  filled_image = binary_fill_holes(dilated_image)

  seg_n_image = label(filled_image)
  plt.imshow(seg_n_image)
  plt.title('Nuclei segmentation')
  plt.show()

  #Loading in and pre-processing images above. Below, we are finding the centers of each nucleus in the image
  #The centers can be input manually by uncommenting the x_centers and y_centers variables below
  #If centers are input manually, be aware that the values you get from FIJI are in (row, column) form, where row
  #corresponds to the y_centers variable and column corresponds to x_centers (I know its not the best naming but here we are)

  distance = ndi.distance_transform_edt(seg_n_image)
  optima_window_size = 75
  coords = peak_local_max(distance,\
                        footprint=np.ones((optima_window_size, optima_window_size)))

  #Need to transpose the coordinates?
  mask = (coords[:, 0] >= 550) & (coords[:, 0] <= 1250) & (coords[:, 1] >= 550) & (coords[:, 1] <= 1250)
  print(coords)
  center_coords = coords[mask]
  center_original = np.mean(center_coords, axis=0)
  center = center_original[::-1]
  x_centers.append(int(center[1]))
  y_centers.append(int(center[0]))


  plt.imshow(distance)
  plt.title('Distance transform of nuclei segmentation')
  plt.show()
  # Display the image
  plt.imshow(seg_n_image)
  plt.scatter(y_centers[len(y_centers)-1], x_centers[len(x_centers)-1])
  plt.title("Centroid of the Object")
  plt.show()

#of course, you can also manually enter the x y values for the center of nuclei by uncommenting the codes below
#x_centers[0] = 250
#y_centers[0] = 250
#plt.imshow(nuclei_im[0])
#plt.scatter(y_centers[0], x_centers[0])
#plt.show()

## Automatically Calculating Nuclei Center for Confocal only

In [ ]:
y_centers = []
x_centers = []
for image in nuclei_im:
  im = imAdjust(image)

  n_image_mask = cv2.GaussianBlur(im,(7,7),0)
  plt.imshow(n_image_mask, cmap = 'gray')
  plt.title('Blurred nuclei image')
  plt.show()
  threshold = 20
  bin_n_image = n_image_mask > threshold

  dilated_image = binary_dilation(bin_n_image, structure=np.ones((3, 3)))
  filled_image = binary_fill_holes(dilated_image)

  seg_n_image = label(filled_image)
  plt.imshow(seg_n_image)
  plt.title('Nuclei segmentation')
  plt.show()

  #Loading in and pre-processing images above. Below, we are finding the centers of each nucleus in the image
  #The centers can be input manually by uncommenting the x_centers and y_centers variables below
  #If centers are input manually, be aware that the values you get from FIJI are in (row, column) form, where row
  #corresponds to the y_centers variable and column corresponds to x_centers (I know its not the best naming but here we are)

  distance = ndi.distance_transform_edt(seg_n_image)
  optima_window_size = 25
  coords = peak_local_max(distance,\
                        footprint=np.ones((optima_window_size, optima_window_size)))

  #Need to transpose the coordinates?
  mask = (coords[:, 0] >= 30) & (coords[:, 0] <= 70) & (coords[:, 1] >= 30) & (coords[:, 1] <= 70)
  center_coords = coords[mask]
  center_original = np.mean(center_coords, axis=0)
  center = center_original[::-1]
  x_centers.append(int(center[1]))
  y_centers.append(int(center[0]))


  plt.imshow(distance)
  plt.title('Distance transform of nuclei segmentation')
  plt.show()
  # Display the image
  plt.imshow(seg_n_image)
  plt.scatter(y_centers[len(y_centers)-1], x_centers[len(x_centers)-1])
  plt.title("Centroid of the Object")
  plt.show()

#of course, you can also manually enter the x y values for the center of nuclei by uncommenting the codes below
#x_centers[0] = 250
#y_centers[0] = 250
#plt.imshow(nuclei_im[0])
#plt.scatter(y_centers[0], x_centers[0])
#plt.show()

## Concentric Circle Analysis for Airyscans only

note: Depending on the image, the radii_list can be adjusted. Currently it says [x for x in range(30, 600, 30)], where the first number is the inner-most radius, second number is the outermost radius, and the last number means each iteration increment the radius by 30 pixels.

Also, the pixel_threshold variable may be adjusted. A higher threshold value will be less sensitive to microtubule signals.

Drawing concentric circles and calculating MT density. Writing measurements to an excel file named Concentric Circle Data.xlsx

In [ ]:
radii_list = [x for x in range(30, 600, 30)]
df = pd.DataFrame(radii_list, columns = ['radius'])
for num, im in enumerate(micro_im):
  im = imAdjust(im)
  pixel_threshold = 50
  #List to determine radii in pixels for analysis
  total_densities = []
  #for nucleus in range(len(y_centers)): #loop through num of nuclei found in image
  count = 1
  densities = []
  plt.figure(figsize = (15,15))
  for i in range(len(radii_list)): #loop through each radius
      radius = radii_list[i]
      if i == 0:
        #adjust if you adjust the values in the radii list
        prev_rad = 0
      else:
        prev_rad = radii_list[i-1]
      square = im[x_centers[num]-radius:x_centers[num]+radius,y_centers[num]-radius:y_centers[num]+radius].astype(np.float32)
      #cropping circle and ring
      height,width = square.shape
      mask = Image.new('L', [height,width] , 0)
      mask1 = Image.new('L', [height,width] , 0)

      #drawing outer ring
      draw = ImageDraw.Draw(mask)
      draw.pieslice([(0,0), (height,width)], 0, 360, fill = 255, outline = "white")

      #drawing inner ring
      draw = ImageDraw.Draw(mask1)
      diff = radius - prev_rad
      draw.pieslice([(diff,diff), (height - diff,width - diff)], 0, 360, fill = 255, outline = "white")

      #cropping image based on the two rings drawn
      np_mask = np.array(mask)
      p_np_mask = np.array(mask1)
      ring = np.zeros(square.shape)
      for i in range(np_mask.shape[0]):
        for j in range(np_mask.shape[1]):
          if np_mask[i][j] == 255 and p_np_mask[i][j] == 0:
            ring[i][j] = square[i][j]

      #Calculate Density
      microtubules = 0
      total_pixels = 0
      m_im = np.zeros(ring.shape)
      for x in range(ring.shape[0]):
          for y in range(ring.shape[1]):
              if ring[x,y] > pixel_threshold:
                  microtubules += 1
                  m_im[x,y] = ring[x,y]
              if ring[x,y] > 0:
                total_pixels += 1
      plt.subplot(len(radii_list), 3, count)
      plt.imshow(m_im, cmap = 'gray')
      count += 1
      density = microtubules/total_pixels
      densities.append(density)
      #print('density of microtubules at nuclei',nucleus+1,'with radius',radius,'pixels is',density)
  total_densities.append(densities)
  #print('-----------------------------')

  #Printing out and plotting results
  #print(len(total_densities))
  print(total_densities)
  df[file_name[num]] = total_densities[0]
  #df.to_excel(dfo, index = False)
  plt.figure()
  n = 1
  for d in total_densities:
    plt.plot(radii_list, d)
    plt.ylim([0, 1])
    plt.xlabel("Radius away from nuclear center (pixels)")
    plt.ylabel("Microtubule Density")
    print("Desnties at radii", radii_list, "for nucleus", n, "are", d)
    n+=1

  #Displaying the concentric circles onto the image
  im_copy = im.copy()
  for nucleus in range(len(y_centers)):
    for radius in radii_list:
      cv2.circle(im_copy, (y_centers[num], x_centers[num]), radius, (255,255,255), 2)

  plt.figure()
  plt.imshow(im_copy, cmap = 'gray')
  plt.show()
df['Average'] = df.iloc[:, 1:].mean(axis=1)
df.to_excel(dfo, index = False, sheet_name = 'C.C. MT Densities')

dfi = pd.read_excel(dfo)
average = [x for x in list(dfi['Average'])]
plt.plot(radii_list, average)
plt.ylim([0, 1])
plt.xlabel("Radius away from nuclear center (pixels)")
plt.ylabel("Microtubule Density")
plt.title("Average Microtubule Density")
plt.show()


## Concentric Circle Analysis for Confocal only

Note: the pixel_threshold variable may be adjusted. A higher threshold value will be less sensitive to microtubule signals.

In [ ]:
pre_radius_list = [x for x in range(5, 60, 5)]
df = pd.DataFrame(pre_radius_list, columns = ['radius'])
for num, im in enumerate(micro_im):
  max_radius = min(im.shape[0] - x_centers[num], im.shape[1] - y_centers[num], x_centers[num], y_centers[num])
  smaller_dimension = min(im.shape[0], im.shape[1])
  radii_list = [x for x in range(5, max_radius, 5)]
  im = imAdjust(im)
  pixel_threshold = 50
  #List to determine radii in pixels for analysis
  total_densities = []
  #for nucleus in range(len(y_centers)): #loop through num of nuclei found in image
  count = 1
  densities = []
  plt.figure(figsize = (15,15))
  for i in range(len(radii_list)): #loop through each radius
      radius = radii_list[i]
      if i == 0:
        #adjust if you adjust the values in the radii list
        prev_rad = 0
      else:
        prev_rad = radii_list[i-1]
      square = im[x_centers[num]-radius:x_centers[num]+radius,y_centers[num]-radius:y_centers[num]+radius].astype(np.float32)
      #cropping circle and ring
      height,width = square.shape
      mask = Image.new('L', [height,width] , 0)
      mask1 = Image.new('L', [height,width] , 0)

      #drawing outer ring
      draw = ImageDraw.Draw(mask)
      draw.pieslice([(0,0), (height,width)], 0, 360, fill = 255, outline = "white")

      #drawing inner ring
      draw = ImageDraw.Draw(mask1)
      diff = radius - prev_rad
      draw.pieslice([(diff,diff), (height - diff,width - diff)], 0, 360, fill = 255, outline = "white")

      #cropping image based on the two rings drawn
      np_mask = np.array(mask)
      p_np_mask = np.array(mask1)
      ring = np.zeros(square.shape)
      for i in range(np_mask.shape[0]):
        for j in range(np_mask.shape[1]):
          if np_mask[i][j] == 255 and p_np_mask[i][j] == 0:
            ring[i][j] = square[i][j]

      #Calculate Density
      microtubules = 0
      total_pixels = 0
      m_im = np.zeros(ring.shape)
      for x in range(ring.shape[0]):
          for y in range(ring.shape[1]):
              if ring[x,y] > pixel_threshold:
                  microtubules += 1
                  m_im[x,y] = ring[x,y]
              if ring[x,y] > 0:
                total_pixels += 1
      plt.subplot(len(radii_list), 3, count)
      plt.imshow(m_im, cmap = 'gray')
      count += 1
      density = microtubules/total_pixels
      densities.append(density)
      #print('density of microtubules at nuclei',nucleus+1,'with radius',radius,'pixels is',density)
  total_densities.append(densities)
  #print('-----------------------------')

  #Printing out and plotting results
  #print(len(total_densities))
  df[file_name[num]] = pd.Series(total_densities[0])
  #df.to_excel(dfo, index = False)
  plt.figure()
  n = 1
  for d in total_densities:
    plt.plot(radii_list, d)
    plt.ylim([0, 1])
    plt.xlabel("Radius away from nuclear center (pixels)")
    plt.ylabel("Microtubule Density")
    print("Desnties at radii", radii_list, "for nucleus", n, "are", d)
    n+=1

  #Displaying the concentric circles onto the image
  im_copy = im.copy()
  for nucleus in range(len(y_centers)):
    for radius in radii_list:
      cv2.circle(im_copy, (y_centers[num], x_centers[num]), radius, (255,255,255), 1)

  plt.figure()
  plt.imshow(im_copy, cmap = 'gray')
  plt.show()
df['Average'] = df.iloc[:, 1:].mean(axis=1)
df.to_excel(dfo, index = False, sheet_name = 'C.C. MT Densities')
